1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을때, 벡터 데이스베이스에서 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 함께 전달

In [1]:
%pip install -qU pypdf langchain-community


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -qU langchain-text-splitters


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install chromadb
%pip install langchain
%pip install sentence-transformers


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

# 문서 불러오기
#file_path = "text.pdf"
#loader = PyPDFLoader(file_path)
#pages = loader.load()  # 기본 로드 방식 사용

# 문서 쪼개기
# chunk_size: 문서를 쪼개는 단위
# chunk_overlap: 문서를 쪼개는 단위의 겹치는 부분
# 겹치는 부분을 선언하는 이유는 원하는 문서를 가져올 확률을 높이기 위함
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
#chunks = text_splitter.split_documents(pages)

# load_and_split 사용으로 문서 불러오기와 쪼개기를 한 번에 수행
# 분할기 정의
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

# 문서 불러오기와 쪼개기를 한 번에 수행
file_path = "text.pdf"
loader = PyPDFLoader(file_path)
chunks = loader.load_and_split(text_splitter=text_splitter)


# 임베딩
load_dotenv()
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [5]:
from langchain_community.vectorstores import Chroma

# 25.03.25 현재 chroma db는 python 3.13 버전에서 작동하지 않음
# 벡터 데이터베이스 생성
# chroma db는 인 메모리 벡터 데이터베이스
database = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="chroma_db",
    collection_name="tax_collection"
)


In [6]:
# 유사도 검색
query = '연봉 3000만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3)

In [7]:
retrieved_docs

[Document(metadata={'creationdate': '', 'creator': 'PyPDF', 'page': 8, 'page_label': '9', 'producer': 'Aspose.Words for .NET 25.1.0', 'source': 'text.pdf', 'total_pages': 170}, page_content='소득세법\n법제처 9\n 국가법령정보센터\n로서 대통령령으로 정하는 금액\n\u3000\u3000\u30001) 「발명진흥법」 제2조제2호에 따른 종업원등(이하 이 조, 제20조 및 제21조에서 “종업원등”이라 한다\n)이 같은 호에 따른 사용자등(이하 이 조에서 “사용자등”이라 한다)으로부터 받는 보상금. 다만, 보상금을 지\n급한 사용자등과 대통령령으로 정하는 특수관계에 있는 자가 받는 보상금은 제외한다.\n\u3000\u3000\u30002) 대학의 교직원 또는 대학과 고용관계가 있는 학생이 소속 대학에 설치된 「산업교육진흥 및 산학연\n협력촉진에 관한 법률」 제25조에 따른 산학협력단(이하 이 조에서 “산학협력단”이라 한다)으로부터 같은 법 \n제32조제1항제4호에 따라 받는 보상금\n저. 대통령령으로 정하는 복리후생적 성질의 급여\n처. 제20조제1항제6호에 따른 소득 중 다음의 요건을 모두 충족하는 소득으로서 대통령령으로 정하는 금액 \n이하의 금액\n1) 임원 또는 종업원(이하 이 조, 제20조 및 제164조의5에서 “임원등”이라 한다) 본인이 소비하는 것을 \n목적으로 제공받거나 지원을 받아 구입한 재화 또는 용역으로서 대통령령으로 정하는 기간 동안 재\n판매가 허용되지 아니할 것\n2) 해당 재화 또는 용역의 제공과 관련하여 모든 임원등에게 공통으로 적용되는 기준이 있을 것\n4. 연금소득 중 다음 각 목의 어느 하나에 해당하는 소득\n가. 「국민연금법」, 「공무원연금법」 또는 「공무원 재해보상법」, 「군인연금법」 또는 「군인 재해보상법\n」, 「사립학교교직원 연금법」, 「별정우체국법」 또는 「국민연금과 직역

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

# 유사도 검색으로 가져온 문서를 LLM에 질문과 함께 전달
prompt = f"""[Identity]
- 당신은 한국의 소득세 전문가입니다.
- [Context]를 참고해서 사용자의 질문에 대답해주세요.

[Context]
{retrieved_docs}

[Question]
{query}
"""
response = llm.invoke(prompt)

In [9]:
response

AIMessage(content='연봉 3,000만원인 직장인의 소득세를 계산하기 위해서는 근로소득공제를 적용한 후 과세표준을 구하고, 이에 해당하는 세율을 적용해야 합니다. 다음은 대략적인 계산 절차입니다.\n\n1. **근로소득공제**: 연봉을 근로소득으로 보고 일정한 비율로 공제합니다. 소득세법에 따르면 3,000만원의 소득에 대한 근로소득공제는 다음과 같습니다.\n   - 500만원 이하: 전액\n   - 500만원 초과 ~ 1,500만원 이하: 30%\n   - 1,500만원 초과 ~ 4,500만원 이하: 110만원 + 19%\n\n   따라서, 3,000만원의 근로소득에 대한 근로소득공제는:\n   - 1,500만원까지: 500만원 * 100% + 1,000만원 * 30% = 500만원 + 300만원 = 800만원\n   - 나머지 1,500만원에 대해: (3,000만원 - 1,500만원) * 19% = 285만원\n\n   총 근로소득공제액 = 800만원 + 285만원 = 1,085만원\n\n2. **과세표준**: 연봉에서 근로소득공제를 차감하여 과세표준을 구합니다.\n   - 과세표준 = 3,000만원 - 1,085만원 = 1,915만원\n\n3. **소득세 계산**: 과세표준에 소득세율을 적용합니다. 소득세율은 다음과 같습니다 (2023년 기준):\n   - 1,200만원 이하: 6%\n   - 1,200만원 초과 ~ 4,600만원 이하: 72만원 + 초과분의 15%\n\n   따라서, 1,915만원에 대한 소득세는:\n   - 1,200만원까지: 1,200만원 * 6% = 72만원\n   - 나머지 715만원에 대해: 715만원 * 15% = 107.25만원\n\n   따라서, 총 소득세는 72만원 + 107.25만원 = 179.25만원\n\n4. **지방소득세**: 소득세의 10%가 추가로 과세됩니다.\n   - 지방소득세 = 179.25만원 * 10% = 17.925만원\n\n5. **총 세액**: 소득세와 지방소득세를 합산합니다.\n  